In [1]:
file_name='robinho-comments'

In [25]:
import numpy as np
import pandas as pd
import psycopg2
import time
import datetime
import sys
from constants import DB_NAME, DB_HOST, DB_PASS, DB_PORT, DB_USER, DB_TIME

news = sys.argv[1]
news_insert_template = """INSERT INTO news(news_id,title,link,deleted) VALUES({news_id},'{title}','{link}',false);
"""
commnets_insert_template = """INSERT INTO comments(comment_id,content,likes,dislikes,reply_to,news_id,deleted) VALUES({comment_id},'{content}',{likes},{dislikes},'{reply_to}',{news_id},false);
"""
query = ''

def init_db_connection():
    today = datetime.datetime.now()

    try:  # connect to postgres
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            host=DB_HOST,
            port=DB_PORT,
            password=DB_PASS,
            connect_timeout=DB_TIME
        )
        tm = time.time()
        return conn
        print('Connected to database: [%s] @ %d seconds' %
            (DB_HOST, time.time() - tm))
    except Exception as e:
        print('Cannot connect to Postgres [%s]' % DB_HOST)
        print(str(e))
        exit(1)


conn = init_db_connection()
cursor = conn.cursor()

col_names = ['id',
             'content',
             'likes',
             'dislikes',
             'reply_to',
             'news_id',
             'news_title',
             'news_link']
data = pd.read_csv('./collected-data/{}.csv'.format(file_name),sep=';',names=col_names).drop_duplicates()
data

,id,content,likes,dislikes,reply_to,news_id,news_title,news_link
0,953ed958-05b1-499d-8487-3123cf646973,Robinho: estuprador e bolsonaristaE estamos ve...,1,0,NaN,a1f4c69f-7d0f-427f-96d8-6ae04102450a,O assunto 302 caso robinho e violencia contra ...,https://g1.globo.com/podcast/o-assunto/noticia...
1,6611b2ad-e6d0-4c28-98f7-d400662a25c4,PASSARINHO QUE DORME COM MORCEGO ACABA DORMIND...,0,0,NaN,a1f4c69f-7d0f-427f-96d8-6ae04102450a,O assunto 302 caso robinho e violencia contra ...,https://g1.globo.com/podcast/o-assunto/noticia...
2,dabea8ec-e6fc-43a4-8663-c469a30a951b,Eu gostaria que quem defende esse rapaz fosse ...,0,0,NaN,a1f4c69f-7d0f-427f-96d8-6ae04102450a,O assunto 302 caso robinho e violencia contra ...,https://g1.globo.com/podcast/o-assunto/noticia...
3,ed6e836a-3108-4b7d-84d4-493d07fa0be3,Coloca uma pena de 50 anos sem direito a nada....,0,0,NaN,a1f4c69f-7d0f-427f-96d8-6ae04102450a,O assunto 302 caso robinho e violencia contra ...,https://g1.globo.com/podcast/o-assunto/noticia...
4,9cfa1759-8fa1-4929-806d-3005ef87cead,"E o presidiário Suzy, que é estuprador e assas...",0,0,NaN,a1f4c69f-7d0f-427f-96d8-6ae04102450a,O assunto 302 caso robinho e violencia contra ...,https://g1.globo.com/podcast/o-assunto/noticia...
...,...,...,...,...,...,...,...,...
332,83308ba2-2345-48c0-94aa-d3d123bc8a20,Ué e ela está errada por acaso?... ago...,5,0,NaN,a1f4c69f-7d0f-427f-96d8-6ae04102450a,Damares pede cadeia imediatamente para robinho,https://g1.globo.com/politica/noticia/2020/10/...
333,465ebff9-fc48-462a-878e-62ca09dfa455,Calma terra plana. Tem leis a serem seguidas. ...,5,0,NaN,a1f4c69f-7d0f-427f-96d8-6ae04102450a,Damares pede cadeia imediatamente para robinho,https://g1.globo.com/politica/noticia/2020/10/...
334,98835304-c3b2-492b-9f05-1de1c202d7f5,"Isso é pouco, ele tem de ser preso",2,0,Calma terra plana. Tem leis a serem seguidas. ...,a1f4c69f-7d0f-427f-96d8-6ae04102450a,Damares pede cadeia imediatamente para robinho,https://g1.globo.com/politica/noticia/2020/10/...
342,2d8add7b-dce7-410d-896d-f2a4ea2030bf,E o ROBINHO EXXXTUPRADOR ainda foi A FAVOR do ...,4,0,NaN,a1f4c69f-7d0f-427f-96d8-6ae04102450a,Damares pede cadeia imediatamente para robinho,https://g1.globo.com/politica/noticia/2020/10/...


In [26]:
news = data[['news_title','news_link']].drop_duplicates()
news_titles = list(news['news_title'].drop_duplicates())
news_titles

['O assunto 302 caso robinho e violencia contra mulher',
 'Damares pede cadeia imediatamente para robinho']

In [27]:
cursor.execute('select max(news_id) from news;')
news_id = int(cursor.fetchone()[0])
cursor.execute('select max(comment_id) from comments;')
comment_id = int(cursor.fetchone()[0])
cursor.close()
conn.close()
i=0
  

In [28]:
for new in news_titles:
    news_id +=1
    comments = data[data['news_title']==new]
    news_insert = news_insert_template.format(
        news_id=news_id,
        title=str(news[news['news_title']==new]['news_title'].max()),
        link=str(news[news['news_title']==new]['news_link'].max())
    )
    query += news_insert
    news_comment = comments[comments['news_title']==new]
    for index, row in news_comment.iterrows():
        comment_id+=1
        comment_insert = commnets_insert_template.format(
            comment_id=comment_id,
            content=row['content'],
            likes=row['likes'],
            dislikes=row['dislikes'],
            reply_to=row['reply_to'],
            news_id=news_id
        )
        comment_insert = comment_insert.replace("'nan'", 'NULL')
        i+=1
        query += comment_insert
        #fp.write(comment_insert)
